In [ ]:
pip install transformers

In [ ]:
pip install seqeval

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf

# 1. 데이터 로드

In [ ]:
import json
from google.colab import files

uploaded = files.upload()

KeyboardInterrupt: ignored

In [ ]:
ner_label_file = files.upload()

KeyboardInterrupt: ignored

In [ ]:
# 문장과 레이블을 담는 리스트
sentences = []
labels = []

# 파일 불러오기
for i in range(1, 13):
  with open('E2017.NEWS.{0:03d}.json'.format(i)) as data_file:
    local = json.load(data_file)

  # json 파일의 "sentence" 태그 안으로
  locals = local["sentence"]
  # locals 리스트 안에 각 딕션너리를 순서대로
  for locals_index in locals:
    # 각 딕션너리의 "NE" 태그 안으로
    locals_NEs = locals_index["NE"]

    # 한 문장 안에 있는 단어들과 그에 대한 레이블을 담는 리스트
    sentence = []
    label = []
    # locals_NEs 중 하나의 단어와 그에 대한 레이블
    for locals_NE in locals_NEs:
      text = locals_NE['text']
      type_ = locals_NE['type']

      # sentence 리스트 안에 text 추가
      sentence.append(text)
      # label 리스트 안에 type_ 추가
      label.append(type_)

    # sentence 리스트를 sentences 리스트에 추가
    sentences.append(sentence)
    # label 리스트를 labels 리스트에 추가
    labels.append(label)

for sentence in sentences:
  if not sentence:
    del sentence

# train 데이터와 test 데이터 분리
train_data_sentence, test_data_sentence, train_data_label, test_data_label = train_test_split(sentences, labels, test_size=0.2)

print(train_data_sentence)
print(train_data_label)
print(test_data_sentence)
print(test_data_label)

[['최', '학교', '교사'], ['2012년', '23.6%', '2014년', '25.2%'], ['화성-12', '한 달', '5종', '미사일'], ['경찰', '전날', '60여 대', 'TV'], ['김선동', '자유한국당', '원내수석부대표', '뉴시스', '조', '수석'], ['1∼2년 차', '직장인'], ['30분', '77분', '심장'], ['3년', '정', '17일'], ['덕수상고', '행정고시', '입법고시'], ['한 달', '1억'], ['인사추천위', '비서실장', '위원장', '인사수석', '간사'], ['서울'], ['北', 'IRBM', '지대공·지대함 미사일', 'ICBM'], [], [], ['193장', '15장', '검은색', '178장'], ['의료진', '오후 4시 37분', '임', '심장', '혈관', '인공 심박동기', '인공호흡기', '혈액 투석 장치'], ['아이돌', '연습생', '정유라'], ['1학년', '담임교사', '친구'], ['홈택스', '스마트폰', '애플리케이션', '연말'], ['서이종', '서울대', '교수', '신입사원'], ['심근경색', '환자', '임중수', '36', '지난달 23일', '오후 2시 20분', '서울', '신촌', '세브란스병원'], ['30대', '직장인', '신용카드', '전화'], ['합참', '무인기', '북한'], ['2천여만 원', '北', '무인기'], ['경찰', 'CCTV', '4개', '332개', '30% 이상', '한 곳'], ['청와대', '민정수석'], ['지난달 29일', '스커드', '미사일', '명중률'], ['문', '대통령', '두 분', '두 분', '정부'], ['정', '원내수석', '조', '수석'], ['한 달', '2억 9000만 원', '20.7%'], ['근로자'], ['종합주가지수', '코스피', '글로벌 금융 위기', '2085포인트', '2007년 11월 1일', '13%'], ['유사시', '평양'

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')] # label(정답) 파일을 열어서 labels을 가져옴.
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['PS_NAME', 'LC_OTHERS', 'LCP_COUNTRY', 'LCP_PROVINCE', 'LCP_COUNTY', 'LCP_CITY', 'LCP_CAPITALCITY', 'LCG_RIVER', 'LCG_OCEAN', 'LCG_BAY', 'LCG_MOUNTAIN', 'LCG_ISLAND', 'LCG_CONTINENT', 'LC_TOUR', 'LC_SPACE', 'OG_OTHERS', 'OGG_ECONOMY', 'OGG_EDUCATION', 'OGG_MILITARY', 'OGG_MEDIA', 'OGG_SPORTS', 'OGG_ART', 'OGG_MEDICINE', 'OGG_RELIGION', 'OGG_SCIENCE', 'OGG_LIBRARY', 'OGG_LAW', 'OGG_POLITICS', 'OGG_FOOD', 'OGG_HOTEL', 'AF_CULTURAL_ASSET', 'AF_BUILDING', 'AF_MUSICAL_INSTRUMENT', 'AF_ROAD', 'AF_WEAPON', 'AF_TRANSPORT', 'AF_WORKS', 'AFW_DOCUMENT', 'AFW_PERFORMANCE', 'AFW_VIDEO', 'AFW_ART_CRAFT', 'AFW_MUSIC', 'AF_WARES', 'DT_OTHERS', 'DT_DURATION', 'DT_DAY', 'DT_MONTH', 'DT_YEAR', 'DT_SEASON', 'DT_GEOAGE', 'DT_DYNASTY', 'TI_OTHERS', 'TI_DURATION', 'TI_HOUR', 'TI_MINUTE', 'TI_SECOND', 'CV_NAME', 'CV_TRIBE', 'CV_SPORTS', 'CV_SPORTS_INST', 'CV_POLICY', 'CV_TAX', 'CV_FUNDS', 'CV_LANGUAGE', 'CV_BUILDING_TYPE', 'CV_FOOD', 'CV_DRINK', 'CV_CLOTHING', 'CV_POSITION', 'CV_RELATION', 'CV_OC

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)} # 태그를 인덱스로 표현하는 변수
index_to_tag = {index: tag for index, tag in enumerate(labels)} # 인덱스를 태그로 표현하는 변수

In [ ]:
tag_size = len(tag_to_index) # 태그의 개수
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 146


# 2. 전처리 예시

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base") # BertTokenizer 에서 이미 학습되어 있는 'klue/bert-base' 모델을 가져옴.

# 3. 전처리

In [ ]:
#? 샘플을 특징으로 변환
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token # cls_token : BERT가 분류 문제를 풀기 위해 추가된 특별 토큰. 모든 문장의 가장 첫 번째(문장의 시작) 토큰으로 삽입.
    sep_token = tokenizer.sep_token # sep_token : 각 문장을 구분하기 위해 사용되는 특별 토큰. 각 문장의 끝에 [SEP] 토큰이 사용됨.
    pad_token_id = tokenizer.pad_token_id # pad_token : 일정한 크기로 BERT에 입력하기 위해 사용되는 패딩 토큰.

    #? input_ids : input의 id를 달아주기 위한 변수.
    # attention_masks : 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록 실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 시퀀스 입력. 실제 단어 및 토큰 : 1 / 패딩 토큰 : 0 으로 표현.
    #? token_type_ids : 토큰의 타입에 대한 id.
    # data_labels : 데이터에 대한 레이블 집합.
    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    # tqdm() : 진행 상황을 Progress Bar 형태로 확인 가능.
      # total : 총 반복 횟수
    # zip() : 두 리스트를 튜플 리스트로 합침.
    '''
        ex) for number, upper, lower in zip("12345", "ABCDE", "abcde"):
                print(number, upper, lower)

            (1, A, a)
            (2, B, b)
            (3, C, c)
            .
            .
            .
    '''
    for example, label in tqdm(zip(examples, labels), total=len(examples)):
        # tokens, labels_ids
        tokens = []
        labels_ids = []

        # example 과 label 에서 단어 하나와 그 단어에 해당하는 label_token 을 가져옴. 
        for one_word, label_token in zip(example, label):
            # one_word 에서 토큰화 작업을 하여 subword_token 들로 분리.
            print()
            print(one_word)
            subword_tokens = tokenizer.tokenize(one_word)
            print(subword_tokens)

            # list.append(['Ping', 'Pong']) : 리스트 끝에 iterable 자체 그대로 넣음.
              # ex) list: ['Tick', 'Tock', 'Song', ['Ping', 'Pong']]
            # list.extend(['Ping', 'Pong']) : 리스트 끝에 iterable의 모든 항목을 넣음.
              # ex) list: ['Tick', 'Tock', 'Song', 'Ping', 'Pong']
            tokens.extend(subword_tokens)

            # labels_ids.extend() : 리스트 끝에 iterable의 모든 항목을 넣음.
            labels_ids.extend([tag_to_index[label_token]] + [pad_token_id_for_label] * (len(subword_tokens) - 1))

        # 스페셜 토큰 : [CLS], [SEP]와 같은 토큰들을 의미.
        # 적어도 한 문장에는 [CLS] 1개와 [SEP] 1개가 존재.
        # '[CLS]', '감사합니다', '[SEP]'
        special_tokens_count = 2

        # 토큰의 개수가 최대 seq의 개수를 넘겼을 때,
        if len(tokens) > max_seq_len - special_tokens_count:
            # tokens 를 최대 seq의 개수(special_tokens의 개수를 포함하여)만큼 인덱스 슬라이싱.
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            # labels_ids 를 최대 seq의 개수(special_tokens의 개수를 포함하여)만큼 인덱스 슬라이싱. 
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        # 한 문장이 다 들어갔으므로, tokens 에 [sep_token]([SEP]) 을 추가.
        tokens += [sep_token]
        # sep_token 에 해당하는 id 추가.
        labels_ids += [pad_token_id_for_label]

        # 모든 문장의 앞에 존재하여 하므로, tokens 앞에 [cls_token]([CLS]) 을 추가.
        tokens = [cls_token] + tokens
        # cls_token 에 해당하는 id 추가.
        labels_ids = [pad_token_id_for_label] + labels_ids

        # tokens 를 ids 로 변환한 것은 input_id 에 넣음.
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        # 어텐션 마스크란?
        ''' 
            BERT가 어텐션 연산을 할 때, 불필요하게 패딩 토큰에 대해서 어텐션을 하지 않도록
            실제 단어와 패딩 토큰을 구분할 수 있도록 알려주는 입력.
            필기 17-02-10 참고.
        '''
        # input_id 길이만큼 attention_mask 에 1을 추가.
        attention_mask = [1] * len(input_id)
        # max_seq_len 보다 input_id 의 개수가 더 적은 경우, 패딩해야 하므로,
        padding_count = max_seq_len - len(input_id)
        # 패딩하기 위해서 padding id 를 padding_count 만큼 input_id 에 채워넣음.
        input_id = input_id + ([pad_token_id] * padding_count)
        # 어텐션 마스크에는 padding_count 만큼 0을 추가.
        attention_mask = attention_mask + ([0] * padding_count)
        #? token_type_id 에 max_seq_len 만큼 pad_token_id_for_segment 를 넣음.
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        #? label 은 현재까지의 labels_ids 에 padding_count 만큼 pad_token_id_for_label 을 넣음.
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        # assert 는 뒤의 조건이 True 가 아니면 AssertError 를 발생.
        # input_id, attention_mask, token_type_id, label 의 크기가 max_seq_len 과 같으면, 에러를 발생.
        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        # input_ids 리스트에 input_id 리스트를 추가.
        input_ids.append(input_id)
        # attention_masks 리스트에 attention_mask 리스트를 추가.
        attention_masks.append(attention_mask)
        # token_type_ids 리스트에 token_type_id 리스트를 추가.
        token_type_ids.append(token_type_id)
        # data_labels 리스트에 label 리스트를 추가.
        data_labels.append(label)
    # 반복문 종료

    # input_ids, attention_masks, token_type_ids 리스트를 np.array() 로 새롭게 만듦.
    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    # np.asarray() 는 데이터 형태가 다를 때만 복사. 데이터 형태가 같을 때 복사하지 않음.
    #? data_labels 리스트를 np.asarray() 로 새롭게 만듦. 데이터 타입을 np.int32 로 새롭게 만듦.
    data_labels = np.asarray(data_labels, dtype=np.int32)

    # 전처리를 수행한 결과를 리턴.
    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
# 위에서 만든 전처리 함수를 통해 훈련용 데이터를 가공.
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

 23%|██▎       | 58/256 [00:00<00:00, 514.30it/s]


최
['최']

학교
['학교']

교사
['교사']

2012년
['2012', '##년']

23.6%
['23', '.', '6', '%']

2014년
['2014', '##년']

25.2%
['25', '.', '2', '%']

화성-12
['화성', '-', '12']

한 달
['한', '달']

5종
['5', '##종']

미사일
['미사일']

경찰
['경찰']

전날
['전날']

60여 대
['60', '##여', '대']

TV
['TV']

김선동
['김선', '##동']

자유한국당
['자유', '##한', '##국', '##당']

원내수석부대표
['원내', '##수', '##석', '##부대', '##표']

뉴시스
['뉴', '##시스']

조
['조']

수석
['수석']

1∼2년 차
['1', '##∼', '##2', '##년', '차']

직장인
['직장인']

30분
['30', '##분']

77분
['77', '##분']

심장
['심장']

3년
['3', '##년']

정
['정']

17일
['17', '##일']

덕수상고
['덕수', '##상', '##고']

행정고시
['행정', '##고시']

입법고시
['입법', '##고시']

한 달
['한', '달']

1억
['1', '##억']

인사추천위
['인사', '##추', '##천', '##위']

비서실장
['비서실', '##장']

위원장
['위원장']

인사수석
['인사', '##수', '##석']

간사
['간사']

서울
['서울']

北
['北']

IRBM
['IR', '##BM']

지대공·지대함 미사일
['지대', '##공', '·', '지대', '##함', '미사일']

ICBM
['ICBM']

193장
['193', '##장']

15장
['15', '##장']

검은색
['검은색']

178장
['178', '##장']

의료진
['의료진']

오후 4시 37분
['오후', '4', '##시', '37', '##분']

임


 63%|██████▎   | 162/256 [00:00<00:00, 504.45it/s]

['청와대']

조
['조']

수석
['수석']

임
['임']

갈비뼈
['갈비', '##뼈']

6~7개
['6', '~', '7', '##개']

1~2도
['1', '~', '2', '##도']

화상
['화상']

미국
['미국']

토요일
['토요일']

오전 2시
['오전', '2', '##시']

상담원
['상담원']

한국
['한국']

신용카드
['신용카드']

열 단계
['열', '단계']

상담원
['상담원']

2학년
['2', '##학년']

경제
['경제']

송
['송']

교사
['교사']

아이돌
['아이돌']

연습생
['연습생']

정유라
['정유라']

후보자
['후보자']

대한민국
['대한민국']

경찰
['경찰']

관계자
['관계자']

목격자
['목격자']

3명
['3', '##명']

CCTV
['CCTV']

니콘
['니', '##콘']

D800
['D', '##80', '##0']

카메라
['카메라']

니콘
['니', '##콘']

35㎜
['35', '##㎜']

26일
['26', '##일']

부동산114
['부동산', '##11', '##4']

이번 주
['이번', '주']

서울
['서울']

1주일 전
['1', '##주', '##일', '전']

0.3%
['0', '.', '3', '%']

작년 10월 이후
['작년', '10', '##월', '이후']

무인기
['무인기']

중앙처리장치
['중앙', '##처리', '##장치']

CPU
['CPU']

카메라
['카메라']

소형컴퓨터
['소형', '##컴', '##퓨', '##터']

최
['최']

체육특기자
['체육', '##특', '##기', '##자']

담당 교사
['담당', '교사']

아빠
['아빠']

정윤회
['정윤', '##회']

10대
['10', '##대']

중국
['중국']

건설은행
['건설', '##은행']

미국
['미국']

뱅크오브아메리카
['뱅크', '##오', '##브', '##아', '#

100%|██████████| 256/256 [00:00<00:00, 502.62it/s]

['콘', '##택', '##트', '##렌', '##즈']

황
['황']

교사
['교사']

북한
['북한']

지난 4월 26일
['지난', '4', '##월', '26', '##일']

무인기
['무인기']

성주
['성주']

전문가
['전문가']

대선
['대선']

조
['조']

수석
['수석']

국회 운영위원회
['국회', '운영', '##위원회']

5분
['5', '##분']

핵
['핵']

사드
['사드']

THAAD
['THAAD']

하나
['하나']

정
['정']

실장
['실장']

적임자
['적임자']

국민
['국민']

맞벌이 부부
['맞벌이', '부부']

근로자
['근로자']

건설사
['건설사']

협의체
['협의', '##체']

운영위원
['운영', '##위원']

건설사
['건설사']

임원
['임원']

징역 2년
['징역', '2', '##년']

임원
['임원']

18명
['18', '##명']

징역 8월
['징역', '8', '##월']

2년
['2', '##년']

집행유예
['집행유예']

1∼3년
['1', '##∼', '##3', '##년']

1학년
['1', '##학년']

과학
['과학']

오
['오']

교사
['교사']

한 학기
['한', '학기']

1~2번
['1', '~', '2', '##번']

조
['조']

수석
['수석']

비서실장
['비서실', '##장']

임
['임']

1주일
['1', '##주', '##일']

4대강
['4', '##대', '##강']

담합 비리
['담합', '비리']

과징금
['과징금']

2중
['2', '##중']

3중
['3', '##중']

북한
['북한']

중앙TV
['중앙', '##TV']

사드
['사드']

성주
['성주']

북쪽
['북쪽']

X 대역탐지기
['X', '대역', '##탐', '##지기']

서쪽
['서쪽']

배우자
['배우자']

난임
['난', '##임']

9번
['9', '##번']



In [ ]:
# 위에서 만든 전처리 함수를 통해 테스트용 데이터를 가공.
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 65/65 [00:00<00:00, 577.02it/s]


건설사
['건설사']

지난 2008년
['지난', '2008', '##년']

정부
['정부']

4대강 공사 계획
['4', '##대', '##강', '공사', '계획']

건설사
['건설사']

한국전력공사
['한국전력', '##공사']

관계자
['관계자']

변압기
['변', '##압', '##기']

근로자
['근로자']

신입사원
['신입', '##사원']

조
['조']

수석
['수석']

밤
['밤']

미국
['미국']

10대
['10', '##대']

8개
['8', '##개']

상담원
['상담원']

김동엽
['김동', '##엽']

경남대
['경남대']

극동문제연구소
['극동', '##문제', '##연구소']

교수
['교수']

북한
['북한']

미사일
['미사일']

중국
['중국']

A2/AD(반접근 지역거부)
['A', '##2', '/', 'AD', '(', '반', '##접', '##근', '지역', '##거', '##부', ')']

북한
['북한']

북한
['북한']

열병식
['열병', '##식']

미사일
['미사일']

대륙간탄도미사일(ICBM)
['대륙', '##간', '##탄', '##도', '##미사', '##일', '(', 'ICBM', ')']

1학년
['1', '##학년']

사회
['사회']

김
['김']

교사
['교사']

예체능
['예체능']

학생
['학생']

정
['정']

2014년 3월 24일
['2014', '##년', '3', '##월', '24', '##일']

경기도
['경기도']

파주
['파주']

북한
['북한']

무인기
['무인기']

서울
['서울']

7∼9초
['7', '##∼', '##9', '##초']

서울
['서울']

청와대
['청와대']

지난 3월
['지난', '3', '##월']

한국수자원공사
['한국', '##수', '##자원', '##공사']

4대강
['4', '##대', '##강']

건설사
['건설사']

1차
['1', '##

# 4. 모델링

In [ ]:
class TFBertForTokenClassification(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super(TFBertForTokenClassification, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='classifier')

    def call(self, inputs):
        input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        all_output = outputs[0]
        prediction = self.classifier(all_output)

        return prediction

# 5. 손실 함수

In [ ]:
def compute_loss(labels, logits):

  loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(
            from_logits=True, reduction=tf.keras.losses.Reduction.NONE)
  active_loss = tf.reshape(labels, (-1,)) != -100
  reduced_logits = tf.boolean_mask(tf.reshape(logits, (-1, shape_list(logits)[2])), active_loss)
  labels = tf.boolean_mask(tf.reshape(labels, (-1,)), active_loss)

  return loss_fn(labels, reduced_logits)

# 6. 학습

In [ ]:
# TPU 작동을 위한 코드
device_name = os.environ['COLAB_TPU_ADDR']
TPU_ADDRESS = 'grpc://' + device_name
print('Found TPU at: {}'.format(TPU_ADDRESS))

resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU_ADDRESS)
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

device_name = os.environ['COLAB_TPU_ADDR']
TPU_ADDRESS = 'grpc://' + device_name
print('Found TPU at: {}'.format(TPU_ADDRESS))

Found TPU at: grpc://10.54.46.146:8470
INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.54.46.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.54.46.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


Found TPU at: grpc://10.54.46.146:8470


In [ ]:
with strategy.scope():
  model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=compute_loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])
        
        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):

      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=4, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/4


/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/PartitionedCall_12:1", shape=(512,), dtype=int64), values=Tensor("Adam/gradients/PartitionedCall_12:0", shape=(512, 768), dtype=float32), dense_shape=Tensor("Adam/gradients/PartitionedCall_12:2", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "shape. This may consume a large amount of memory." % value)
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/indexed_slices.py:446: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("Adam/gradients/PartitionedCall_12:7", shape=(512,), dtype=int64), values=Tensor("Adam/gradients/PartitionedCall_12:6", shape=(512, 768), dtype=float32), dense_shape=Tensor("Adam/gradients/PartitionedCall_12:8", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume

6/8 [=====================>........] - ETA: 0s - loss: 0.5768WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0063s vs `on_train_batch_end` time: 7.6895s). Check your callbacks.


8/8 [==============================] - ETA: 0s - loss: 0.5653 - f1: 65.93
                  precision    recall  f1-score   support

        AFW_MUSI       0.00      0.00      0.00         1
     AF_TRANSPOR       0.83      0.83      0.83         6
         AF_WARE       1.00      1.00      1.00         1
        AF_WEAPO       0.43      0.67      0.52         9
          AM_PAR       1.00      1.00      1.00         1
           CV_LA       0.00      0.00      0.00         2
    CV_OCCUPATIO       0.70      0.70      0.70        10
        CV_POLIC       0.00      0.00      0.00         3
      CV_POSITIO       0.76      0.77      0.76        44
      CV_RELATIO       0.50      1.00      0.67         2
           CV_TA       1.00      1.00      1.00         1
           DT_DA       0.50      0.33      0.40         3
      DT_DURATIO       0.00      0.00      0.00         3
         DT_MONT       0.00      0.00      0.00         3
        DT_OTHER       0.36      0.62      0.45        

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: OG_OTHERS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: DT_YEAR seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: OGG_POLITICS seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: QT_COUNT seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:167: UserWarning: OGG_ECONOMY seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/seq

8/8 [==============================] - ETA: 0s - loss: 0.3447 - f1: 67.91
                  precision    recall  f1-score   support

        AFW_MUSI       0.00      0.00      0.00         1
     AF_TRANSPOR       1.00      0.83      0.91         6
         AF_WARE       1.00      1.00      1.00         1
        AF_WEAPO       0.54      0.78      0.64         9
          AM_PAR       1.00      1.00      1.00         1
           CV_LA       0.33      0.50      0.40         2
    CV_OCCUPATIO       0.70      0.70      0.70        10
        CV_POLIC       0.00      0.00      0.00         3
      CV_POSITIO       0.75      0.82      0.78        44
      CV_RELATIO       1.00      1.00      1.00         2
           CV_TA       1.00      1.00      1.00         1
           DT_DA       0.20      0.33      0.25         3
      DT_DURATIO       0.00      0.00      0.00         3
         DT_MONT       0.00      0.00      0.00         3
        DT_OTHER       0.31      0.50      0.38        

# 5. 예측

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer,
                                 pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]

        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask


        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 1400.90it/s]


In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):
  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

In [ ]:
sent1 = '1618∼1648년 독일을 무대로 신교(프로테스탄트)와 구교(가톨릭) 간에 벌어진 종교전쟁.'
sent2 = '25일 오전 6~7시에 걸쳐 한국과 해외 금융회사의 콜센터에 실제로 전화를 걸어 보았더니 한국의 4대 은행은 모두 연결이 안 됐다.'

In [ ]:
test_samples = [sent1, sent2]

In [ ]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 386.32it/s]


In [ ]:
result_list

[[('1618∼1648년', 'DT_OTHERS'),
  ('독일을', 'LCP_COUNTRY'),
  ('무대로', 'LCP_COUNTY'),
  ('신교(프로테스탄트)와', 'QT_PERCENTAGE'),
  ('구교(가톨릭)', 'PS_NAME'),
  ('간에', 'CV_CURRENCY'),
  ('벌어진', 'TM_DIRECTION'),
  ('종교전쟁.', 'DT_SEASON')],
 [('25일', 'DT_DAY'),
  ('오전', 'TI_DURATION'),
  ('6~7시에', 'QT_COUNT'),
  ('걸쳐', 'OGG_EDUCATION'),
  ('한국과', 'LCP_COUNTRY'),
  ('해외', 'LCP_COUNTRY'),
  ('금융회사의', 'OGG_ECONOMY'),
  ('콜센터에', 'OGG_ECONOMY'),
  ('실제로', 'TMI_HW'),
  ('전화를', 'TMI_HW'),
  ('걸어', 'TMI_HW'),
  ('보았더니', 'AFW_VIDEO'),
  ('한국의', 'LCP_COUNTRY'),
  ('4대', 'QT_COUNT'),
  ('은행은', 'CV_OCCUPATION'),
  ('모두', 'DT_DURATION'),
  ('연결이', 'OGG_POLITICS'),
  ('안', 'TR_SOCIAL_SCIENCE'),
  ('됐다.', 'DT_MONTH')]]

In [ ]:
from google.cloud import storage

bucket_name = 'jibdolee'

bucket = storage_client.get_bucket(bucket_name)

NameError: ignored

In [ ]:
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='mindful-span-350510-023cbf6de201.json'

save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')
# model.save('gs://jibdolee/my_model', options=save_locally)
model.save('my_model', options=save_locally)

INFO:tensorflow:Assets written to: my_model/assets


INFO:tensorflow:Assets written to: my_model/assets


In [ ]:
with strategy.scope():
  new_model = TFBertForTokenClassification("klue/bert-base", num_labels=tag_size)
  new_optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  new_model.compile(optimizer=new_optimizer, loss=compute_loss)

load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
new_model = tf.keras.models.load_model('my_model', options=load_locally)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

ValueError: ignored

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-8yy2n012
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-8yy2n012
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)
